<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/06wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [ ]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-wcPiCEdYML9-6-Xv5RVbso&si=BbNo6mwCHqwOV0FS>}}

# 2. Imports

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [3]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. CNN 자랑

## A. 성능좋음

*Fashion MNIST*

In [47]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset]).to("cuda:0")
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float().to("cuda:0")
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset]).to("cuda:0")
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float().to("cuda:0")

*발악수준으로 설계한 신경망*

In [56]:
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [49]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [50]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

tensor(1., device='cuda:0')

In [51]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

tensor(0.8520, device='cuda:0')

*대충대충 설계한 합성곱신경망*

In [52]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [53]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [54]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

tensor(0.9466, device='cuda:0')

In [55]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

tensor(0.8710, device='cuda:0')

## B. 파라메터적음

In [76]:
net1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
)
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
)

In [77]:
net1_params = list(net1.parameters())
net2_params = list(net2.parameters())

In [78]:
for params in net1_params:
    print(params.shape)

torch.Size([2048, 784])
torch.Size([2048])
torch.Size([10, 2048])
torch.Size([10])


In [79]:
2048*784 + 2048 + 10*2048 + 10 

1628170

In [80]:
for params in net2_params:
    print(params.shape)

torch.Size([16, 1, 2, 2])
torch.Size([16])
torch.Size([10, 2704])
torch.Size([10])


In [81]:
16*1*2*2 + 16 + 10*2704 + 10

27130

## C. 유명함

`-` <https://brunch.co.kr/@hvnpoet/109> 

# 4. CNN 핵심레이어 

## A. `torch.nn.ReLU`

**(예시1) 연산방법**

In [126]:
img = torch.randn(1,1,5,5) # 흑백이미지한장
relu = torch.nn.ReLU()

In [127]:
img # - 값도 있지만 이미지라고 생각하자..

tensor([[[[ 1.7009,  0.7528,  0.1870,  0.2400, -0.3679],
          [ 2.6662,  1.3107,  0.9368, -0.8709, -1.7125],
          [-1.6364, -0.8744,  0.6034,  0.0990,  1.7091],
          [ 0.2130, -0.0287, -0.2431,  1.7716,  1.1965],
          [-0.3508, -0.1929,  1.1716, -0.6615,  1.3645]]]])

In [128]:
relu(img)

tensor([[[[1.7009, 0.7528, 0.1870, 0.2400, 0.0000],
          [2.6662, 1.3107, 0.9368, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.6034, 0.0990, 1.7091],
          [0.2130, 0.0000, 0.0000, 1.7716, 1.1965],
          [0.0000, 0.0000, 1.1716, 0.0000, 1.3645]]]])

## B. `torch.nn.MaxPool2d`

**(예시1) 연산방법, kernel_size 의 의미**

In [172]:
img = torch.rand(1,1,4,4) # 흑백이미지한장
mp = torch.nn.MaxPool2d(kernel_size=2)

In [173]:
img

tensor([[[[0.9650, 0.4934, 0.3260, 0.4669],
          [0.2401, 0.5172, 0.3875, 0.9785],
          [0.7401, 0.5297, 0.3935, 0.2731],
          [0.3243, 0.1672, 0.3824, 0.5780]]]])

In [174]:
mp(img)

tensor([[[[0.9650, 0.9785],
          [0.7401, 0.5780]]]])

**(예시2) 이미지크기와 딱 맞지않는 커널일경우?**

In [71]:
img = torch.rand(1,1,5,5) # 흑백이미지한장
mp = torch.nn.MaxPool2d(kernel_size=2)

In [72]:
img

tensor([[[[0.8015, 0.9923, 0.5175, 0.2965, 0.7411],
          [0.8369, 0.2625, 0.9127, 0.2559, 0.1240],
          [0.8461, 0.2436, 0.4713, 0.5677, 0.8162],
          [0.0301, 0.2276, 0.2605, 0.6200, 0.1242],
          [0.1616, 0.7613, 0.6406, 0.2210, 0.2042]]]])

In [73]:
mp(img)

tensor([[[[0.9923, 0.9127],
          [0.8461, 0.6200]]]])

**(예시3) 정사각형이 아닌 커널**

In [61]:
img = torch.rand(1,1,4,4) # 흑백이미지한장
mp = torch.nn.MaxPool2d(kernel_size=(2,4))

In [62]:
img

tensor([[[[0.2001, 0.1130, 0.2961, 0.9473],
          [0.2944, 0.8712, 0.8057, 0.4017],
          [0.0661, 0.9816, 0.4964, 0.0813],
          [0.5853, 0.0936, 0.1827, 0.2528]]]])

In [63]:
mp(img)

tensor([[[[0.9473],
          [0.9816]]]])

## C. `torch.nn.Conv2d`

**(예시1) 연산방법, stride=2**

In [89]:
img = torch.rand(1,1,4,4) # 흑백이미지한장
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=2)

In [90]:
img

tensor([[[[0.6455, 0.0018, 0.8217, 0.7056],
          [0.7984, 0.5098, 0.4589, 0.6528],
          [0.3544, 0.5795, 0.1119, 0.8301],
          [0.0376, 0.8687, 0.2402, 0.8886]]]])

In [91]:
conv(img)

tensor([[[[0.6272, 0.8083],
          [0.2748, 0.4018]]]], grad_fn=<ConvolutionBackward0>)

??

In [114]:
conv.weight, conv.bias

(Parameter containing:
 tensor([[[[ 0.3506,  0.4990],
           [ 0.4772, -0.4890]]]], requires_grad=True),
 Parameter containing:
 tensor([0.2682], requires_grad=True))

In [168]:
(img[:,  :,  :2,  :2] * conv.weight.data).sum() + conv.bias, conv(img)

(tensor([1.0690], grad_fn=<AddBackward0>),
 tensor([[[[1.0690, 2.7447],
           [7.7718, 9.4475]]]], grad_fn=<ConvolutionBackward0>))

In [169]:
(img[:,  :,  :2,  2:] * conv.weight.data).sum() + conv.bias, conv(img)

(tensor([2.7447], grad_fn=<AddBackward0>),
 tensor([[[[1.0690, 2.7447],
           [7.7718, 9.4475]]]], grad_fn=<ConvolutionBackward0>))

In [170]:
(img[:,  :,  2:,  :2] * conv.weight.data).sum() + conv.bias, conv(img)

(tensor([7.7718], grad_fn=<AddBackward0>),
 tensor([[[[1.0690, 2.7447],
           [7.7718, 9.4475]]]], grad_fn=<ConvolutionBackward0>))

In [171]:
(img[:,  :,  2:,  2:] * conv.weight.data).sum() + conv.bias, conv(img)

(tensor([9.4475], grad_fn=<AddBackward0>),
 tensor([[[[1.0690, 2.7447],
           [7.7718, 9.4475]]]], grad_fn=<ConvolutionBackward0>))

**(예시2) 이동평균**

In [228]:
img = torch.arange(1,17).float().reshape(1,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,bias=False)
conv.weight.data = conv.weight.data*0+1/4

In [230]:
img

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [231]:
conv(img)

tensor([[[[ 3.5000,  4.5000,  5.5000],
          [ 7.5000,  8.5000,  9.5000],
          [11.5000, 12.5000, 13.5000]]]], grad_fn=<ConvolutionBackward0>)

**(예시3) 2개의 이미지**

In [232]:
imgs = torch.arange(1,33).float().reshape(2,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,bias=False)
conv.weight.data = conv.weight.data*0+1/4

In [234]:
imgs

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]],


        [[[17., 18., 19., 20.],
          [21., 22., 23., 24.],
          [25., 26., 27., 28.],
          [29., 30., 31., 32.]]]])

In [235]:
conv(imgs)

tensor([[[[ 3.5000,  4.5000,  5.5000],
          [ 7.5000,  8.5000,  9.5000],
          [11.5000, 12.5000, 13.5000]]],


        [[[19.5000, 20.5000, 21.5000],
          [23.5000, 24.5000, 25.5000],
          [27.5000, 28.5000, 29.5000]]]], grad_fn=<ConvolutionBackward0>)

**(예시4) 2개의 out_channels**

In [257]:
img = torch.arange(1,33).float().reshape(2,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=2,kernel_size=2,stride=1,bias=False)
conv.weight.data[0] = conv.weight.data[0]*0 + 1/4
conv.weight.data[1] = conv.weight.data[1]*0 

In [258]:
img

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]],


        [[[17., 18., 19., 20.],
          [21., 22., 23., 24.],
          [25., 26., 27., 28.],
          [29., 30., 31., 32.]]]])

In [259]:
conv(img)

tensor([[[[ 3.5000,  4.5000,  5.5000],
          [ 7.5000,  8.5000,  9.5000],
          [11.5000, 12.5000, 13.5000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[19.5000, 20.5000, 21.5000],
          [23.5000, 24.5000, 25.5000],
          [27.5000, 28.5000, 29.5000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]]], grad_fn=<ConvolutionBackward0>)